# Calculating Elder Impulse for long view

In [1]:
# IMPORTING PACKAGES

import pandas as pd
import requests
import pandas_ta as ta
#import matplotlib.pyplot as plt
#from termcolor import colored as cl
#import math 
from datetime import datetime, timedelta
import sqlite3

#plt.rcParams['figure.figsize'] = (20,10)
#plt.style.use('fivethirtyeight')

## Getting historical data from binance API

In [2]:
#The variable days_back represents the number of days in the past from which I intend to retrieve the data.
#The variable symbol is a coin pair. For example if you want bitcoin historical data, the symbol to use is BTCUSDT. 

API_URL = 'https://api.binance.com/api/v3/klines' # Binance API endpoint for candlestick data

def get_historical_data(symbol, days_back, interval):
    # Calculate the start time in milliseconds
    start_time = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)

    # Calculate the current time in milliseconds
    end_time = int(datetime.now().timestamp() * 1000)

    # Make an API request to retrieve historical data
    hist_json = requests.get(API_URL, params={
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Maximum number of data points per request
    }).json()

    #print(hist_json)

    # Making a df from response
    # Extract only the relevant elements for each row
    df = pd.DataFrame(hist_json, columns=[
        'Time', 'Open Price', 'High Price', 'Low Price',
        'Close Price', 'Volume', 'Kline Close Time', 'Quote Asset Volume',
        'Number of Trades', 'Taker Buy Base Asset Volume',
        'Taker Buy Quote Asset Volume', 'Unused Field'
    ])

    # Select only the relevant columns (Open Time, Open Price, High Price, Low Price, Close Price, Volume)
    df = df[['Time', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']]
    df['dateTime'] = pd.to_datetime(df['Time'], unit='ms')
    
    return df


## creating a table for calculating elder impulse

Elder Impulse
https://school.stockcharts.com/doku.php?id=chart_analysis:elder_impulse_system
MACD-histogram
https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp

In [4]:
df_long = get_historical_data('BTCUSDT', 30, '1w')


In [5]:
df_long.tail()


,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime
0,1705276800000,41732.35000000,43578.01000000,40280.00000000,41580.33000000,238486.27274000,2024-01-15
1,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22
2,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29
3,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05
4,1707696000000,48300.00000000,50368.61000000,47710.01000000,49178.53000000,108182.35760000,2024-02-12


In [6]:
df_long['13EMA'] = df_long['Close Price'].ewm(span=13, adjust=False).mean()

In [7]:
df_long.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA
0,1705276800000,41732.35000000,43578.01000000,40280.00000000,41580.33000000,238486.27274000,2024-01-15,41580.330000
1,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22,41644.720000
2,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29,41778.742857
3,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05,42710.349592
4,1707696000000,48300.00000000,50368.61000000,47710.01000000,49178.53000000,108182.35760000,2024-02-12,43634.375364


In [8]:
# Calculate 12-period EMA
df_long['12EMA'] = df_long['Close Price'].ewm(span=12, adjust=False).mean()

# Calculate 26-period EMA
df_long['26EMA'] = df_long['Close Price'].ewm(span=26, adjust=False).mean()

# Calculate MACD Line (12EMA - 26EMA)
df_long['MACD'] = df_long['12EMA'] - df_long['26EMA']

# Calculate Signal Line (9-period EMA of MACD)
df_long['Signal'] = df_long['MACD'].ewm(span=9, adjust=False).mean()

# Calculate MACD Histogram
df_long['Histogram'] = df_long['MACD'] - df_long['Signal']

In [9]:
df_long.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA,12EMA,26EMA,MACD,Signal,Histogram
0,1705276800000,41732.35000000,43578.01000000,40280.00000000,41580.33000000,238486.27274000,2024-01-15,41580.330000,41580.330000,41580.330000,0.000000,0.000000,0.000000
1,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22,41644.720000,41649.673077,41613.717407,35.955670,7.191134,28.764536
2,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29,41778.742857,41793.243373,41685.507229,107.736144,27.300136,80.436008
3,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05,42710.349592,42794.281315,42175.468916,618.812400,145.602589,473.209811
4,1707696000000,48300.00000000,50368.61000000,47710.01000000,49178.53000000,108182.35760000,2024-02-12,43634.375364,43776.473421,42694.214181,1082.259239,332.933919,749.325321


## Calculating elder impulse

In [10]:
last_row_13ema = df_long.iloc[-1]['13EMA']
second_to_last_row_13ema = df_long.iloc[-2]['13EMA']
# Print the comparison
print(f"Last row EMA_13: {last_row_13ema}")
print(f"Second-to-last row EMA_13: {second_to_last_row_13ema}")

Last row EMA_13: 43634.37536443149
Second-to-last row EMA_13: 42710.34959183674


In [11]:
ema_comparison = 0

if last_row_13ema > second_to_last_row_13ema:
    ema_comparison = 1
elif last_row_13ema < second_to_last_row_13ema:
    ema_comparison = -1

ema_comparison

1

In [12]:
elder_impulse = 0

if ema_comparison == 1 and df_long['Histogram'].iloc[-1] > 0:
    elder_impulse = 1
elif ema_comparison == -1 and df_long['Histogram'].iloc[-1] < 0:
    elder_impulse = -1
else:
    elder_impulse = 0

elder_impulse

1

if elder_impulse is 1, it means you can go long; 
if it is -1 , you can go short;
if it is 0 you can do nothing

# Creating Medium view and ATR channels

In [ ]:
df_medium = get_historical_data('BTCUSDT', 30, '1w')